# Очистка признаков и тренировка пробных моделей
В качестве основы взят блокнот: [link](https://colab.research.google.com/drive/12-3FFYrJVH2xtzH1W_OVbwp0Yw89wfa1?usp=sharing)

## Импорты

In [1]:
# !pip install catboost

In [2]:
import pandas as pd
import numpy as np

import sys 
import os
sys.path.append(os.path.abspath("Data_preproc"))
from pos_prepr_by_Igor import *

## Загрузка датасета

In [3]:
df = load_data()

In [4]:
df.head(5)

,BirthDate,education,employment status,Value,JobStartDate,Position,MonthProfit,MonthExpense,Gender,Family status,...,SNILS,BankA_decision,BankB_decision,BankC_decision,BankD_decision,BankE_decision,Merch_code,Loan_amount,Loan_term,Goods_category
SkillFactory_Id,,,,,,,,,,,,,,,,,,,,,
1.0,1988-07-21,Высшее - специалист,Работаю по найму полный рабочий день/служу,9 - 10 лет,2013-09-01,начальник п,180000.0,90000.0,0.0,Никогда в браке не состоял(а),...,0.0,success,success,success,success,denied,77.0,137000.0,18.0,Furniture
2.0,1998-01-16,Высшее - специалист,Собственное дело,1 - 2 года,2022-12-01,директор,200000.0,60000.0,0.0,Никогда в браке не состоял(а),...,0.0,success,denied,success,denied,success,27.0,33000.0,6.0,Fitness
3.0,1972-10-02,Высшее - специалист,Работаю по найму полный рабочий день/служу,10 и более лет,2005-12-01,ДИРЕКТОР,300000.0,45000.0,1.0,Никогда в браке не состоял(а),...,0.0,denied,denied,success,success,success,19.0,165000.0,24.0,Medical_services
4.0,1972-10-02,Высшее - специалист,Работаю по найму полный рабочий день/служу,10 и более лет,2005-12-01,ДИРЕКТОР,300000.0,45000.0,1.0,Никогда в браке не состоял(а),...,0.0,success,denied,success,success,success,34.0,67000.0,12.0,Education
5.0,1972-10-02,Высшее - специалист,Работаю по найму полный рабочий день/служу,10 и более лет,2005-12-01,ДИРЕКТОР,300000.0,45000.0,1.0,Никогда в браке не состоял(а),...,0.0,success,denied,success,success,success,75.0,36000.0,6.0,Furniture


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6603 entries, 1.0 to 8787.0
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   BirthDate          6603 non-null   datetime64[ns]
 1   education          6603 non-null   object        
 2   employment status  6603 non-null   object        
 3   Value              6603 non-null   object        
 4   JobStartDate       6603 non-null   datetime64[ns]
 5   Position           6603 non-null   object        
 6   MonthProfit        6603 non-null   float64       
 7   MonthExpense       6603 non-null   float64       
 8   Gender             6603 non-null   float64       
 9   Family status      6603 non-null   object        
 10  ChildCount         6603 non-null   float64       
 11  SNILS              6603 non-null   float64       
 12  BankA_decision     6603 non-null   object        
 13  BankB_decision     6603 non-null   object        
 14  BankC_dec

In [6]:
res = test_CarierLevel_feature_creator(df)

res= CarierLevel
НИЗШЕЕ_ЗВЕНО     1809
СРЕДНЕЕ_ЗВЕНО    1302
МЕН1              921
МЕН2              583
ДИР2              554
САМ               521
ИП                404
ДИР1              242
ЗАМДИР            128
РЕЗЕРВ            109
ЗАМНАЧ             30
Name: count, dtype: int64


In [7]:
res = test_ValueDig_feature_creator(df)

Исходное количество nan в фиче <Value>: 0 штук.
Устранено: 0 штук.
Фича <Value> не содержит пропусков.


Легенда по которой выполняется оцифровка трудового стажа:
 {'10 и более лет': 1.0, '3 - 4 года': 0.35, '2 - 3 года': 0.25, '4 - 5 лет': 0.45, '5 - 6 лет': 0.55, '1 - 2 года': 0.15, '6 - 7 лет': 0.65, '7 - 8 лет': 0.75, '8 - 9 лет': 0.85, '6 месяцев - 1 год': 0.075, '9 - 10 лет': 0.95, '4 - 6 месяцев': 0.042, 'менее 4 месяцев': 0.017, '0 месяцев 0 лет': 0.001}


In [8]:
res.ValueDig.value_counts()

ValueDig
1.000    3613
0.350     526
0.250     499
0.450     370
0.550     293
0.150     272
0.650     250
0.750     225
0.850     152
0.075     121
0.001     109
0.950      88
0.042      53
0.017      32
Name: count, dtype: int64

# Кластеризация по доходу

In [9]:
df["Value"][df['Value'].str.contains('0 месяцев 0 лет')]


SkillFactory_Id
163.0     0 месяцев 0 лет
176.0     0 месяцев 0 лет
273.0     0 месяцев 0 лет
413.0     0 месяцев 0 лет
414.0     0 месяцев 0 лет
               ...       
8537.0    0 месяцев 0 лет
8630.0    0 месяцев 0 лет
8671.0    0 месяцев 0 лет
8701.0    0 месяцев 0 лет
8764.0    0 месяцев 0 лет
Name: Value, Length: 109, dtype: object

In [10]:
ValueDig_feature_creator(df)

Исходное количество nan в фиче <Value>: 0 штук.
Устранено: 0 штук.
Фича <Value> не содержит пропусков.
Легенда по которой выполняется оцифровка трудового стажа:
 {'10 и более лет': 1.0, '3 - 4 года': 0.35, '2 - 3 года': 0.25, '4 - 5 лет': 0.45, '5 - 6 лет': 0.55, '1 - 2 года': 0.15, '6 - 7 лет': 0.65, '7 - 8 лет': 0.75, '8 - 9 лет': 0.85, '6 месяцев - 1 год': 0.075, '9 - 10 лет': 0.95, '4 - 6 месяцев': 0.042, 'менее 4 месяцев': 0.017, '0 месяцев 0 лет': 0.001}


SkillFactory_Id
1.0       0.95
2.0       0.15
3.0       1.00
4.0       1.00
5.0       1.00
          ... 
8780.0    1.00
8782.0    1.00
8783.0    1.00
8786.0    0.15
8787.0    0.15
Name: Value, Length: 6603, dtype: float64

In [11]:
from sklearn.cluster import KMeans
import numpy as np

# Загрузка данных
data = np.array([[10000], [20000], [30000], [40000], [50000], [60000]])

# Создание модели k-средних с 2 кластерами
kmeans = KMeans(n_clusters=2)

# Обучение модели
kmeans.fit(data)

# Предсказание кластеров для новых данных
new_data = np.array([[15000], [45000], [55000]])
predicted_clusters = kmeans.predict(new_data)

# Вывод предсказанных кластеров
print(predicted_clusters)

[0 1 1]


/home/igor/anaconda3/envs/cuda_p3_9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## Работа со столбцами

### Обработка признаков BirthDate, education,<br>employment status, Value, JobStartDate<br>(И.Пластов, А.Крупский)
- BirthDate
-	education
- employment status
- Value
- JobStartDate

#### Очистка признаков от пропусков *BirthDate*

In [12]:
df = clean_features(df)

Mode = 0   1998-04-23
Name: BirthDate, dtype: datetime64[ns]
Количество nan в фиче <BirthDate>: 0 штук.
Фича <BirthDate> не содержит пропусков.
Количество nan в фиче <education>: 0 штук.
Фича <education> не содержит пропусков.
Исходное количество nan в фиче <employment status>: 0 штук.
Устранено: 0 штук.
Фича <employment status> не содержит пропусков.
Исходное количество nan в фиче <Value>: 0 штук.
Устранено: 0 штук.
Фича <Value> не содержит пропусков.
<class 'pandas.core.series.Series'>
Исходное количество nan в фиче <JobStartDate>: 0 штук.
Устранено: 0 штук.
Фича <JobStartDate> не содержит пропусков.


In [13]:
df = make_features(df)

Количество nan в фиче <education>: 0 штук.
Фича <education> не содержит пропусков.
Легенда по которой выполняется оцифровка уровня образования:
 {'высшее - специалист': 0.6, 'среднее профессиональное': 0.3, 'среднее': 0.2, 'неоконченное высшее': 0.4, 'бакалавр': 0.5, 'несколько высших': 0.8, 'магистр': 0.7, 'неоконченное среднее': 0.1, 'mba': 0.7, 'ученая степень': 1.0}
Количество nan в фиче <education>: 0 штук.
Фича <education> не содержит пропусков.
Исходное количество nan в фиче <employment status>: 0 штук.
Устранено: 0 штук.
Фича <employment status> не содержит пропусков.
Легенда по которой выполняется оцифровка уровня занятости:
 {'работаю по найму полный рабочий день/служу': 1.0, 'собственное дело': 1.0, 'не работаю': 0.001, 'работаю по найму неполный рабочий день': 0.5, 'студент': 0.001, 'декретный отпуск': 0.001, 'пенсионер': 0.001}
Исходное количество nan в фиче <Value>: 0 штук.
Устранено: 0 штук.
Фича <Value> не содержит пропусков.
Легенда по которой выполняется оцифровка тру

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
def Monthly_credit_payment(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем функцию расчета ежемесячного платёжа'''
    df = df.copy()

    return df['Loan_amount'] // df['Loan_term']

In [ ]:
def Payment_to_income(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем признак "показатель долговой нагрузки"'''
    df = df.copy()
    monthly_payment = Monthly_credit_payment(df) + df['MonthExpense']

    return monthly_payment / df['MonthProfit']

In [ ]:
#!pip install scikit-learn

In [ ]:
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def cat_num_split(df: pd.DataFrame) -> tuple:
    '''Ищем категориальные и числовые признаки в датафрейме'''

    cat_columns = []
    num_columns = []

    for column_name in df.columns:
        if (df[column_name].dtypes == object):
            cat_columns += [column_name]
        else:
            num_columns += [column_name]

    print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))
    print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

    return cat_columns, num_columns


da = pd.read_csv(
    filepath_or_buffer='SF_Mandarin_dataset_ver3_1.csv',
    sep=';',
    index_col='SkillFactory_Id',
    #parse_dates=['BirthDate', 'JobStartDate']  <<------ Формат даты/времени заполнятель не прожевывает
    )


cat_imputer = SimpleImputer(strategy='most_frequent') # Заполнение наиболее частыми значениями для категориальных столбцов
num_imputer = SimpleImputer(strategy='median') # Заполнение медианой для числовых столбцов


cat_columns, num_columns = cat_num_split(da)


da[cat_columns] = cat_imputer.fit_transform(da[cat_columns])
da[num_columns] = num_imputer.fit_transform(da[num_columns])

da.info()

FileNotFoundError: [Errno 2] No such file or directory: 'SF_Mandarin_dataset_ver3_1.csv'

# Самый простой полный алгоритм обучения модели на наших данных

## Импорты

In [ ]:
from datetime import date

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

from catboost import Pool

import matplotlib.pyplot as plt

## Функции


In [ ]:
def cat_num_split(df: pd.DataFrame) -> tuple:
    '''Ищем категориальные и числовые признаки в датафрейме'''

    cat_columns = []
    num_columns = []

    for column_name in df.columns:
        if (df[column_name].dtypes == object):
            cat_columns += [column_name]
        else:
            num_columns += [column_name]

    print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))
    print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

    return cat_columns, num_columns

def Age_feature_creator(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем признак "возраст"'''
    df = df.copy()

    return date.today().year - df['BirthDate'].dt.year


def Num_value_feature_creator(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем признак "стаж работы" из даты начала работы'''
    df = df.copy()

    return date.today().year - df['JobStartDate'].dt.year


def Monthly_credit_payment(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем функцию расчета ежемесячного платёжа'''
    df = df.copy()

    return df['Loan_amount'] // df['Loan_term']


def Payment_to_income(
    df: pd.DataFrame,
    ) -> pd.Series:
    '''Создаем признак "показатель долговой нагрузки"'''
    df = df.copy()
    monthly_payment = Monthly_credit_payment(df) + df['MonthExpense']

    return monthly_payment / df['MonthProfit']


## Загрузка и предобработка


In [ ]:
# Загружаю датафрейм
df = pd.read_csv(
    filepath_or_buffer='SF_Mandarin_dataset_ver3_csv.csv',
    sep=';'#,
#    index_col='SkillFactory_Id',
#    parse_dates=['BirthDate', 'JobStartDate']
    )
df = drop_completely_nan_rows(df) # Пластов
# Создаю признаки "Возраст" и "Стаж работы в виде числа" до заполнения пропусков
# Так как столбцы с датами нужно убивать до заполнения пропусков

""" Пластов
df['Age'] = Age_feature_creator(df)

df['Num_value'] = Num_value_feature_creator(df)

# Убиваю столбцы с датами
# Пластов df.drop(['BirthDate', 'JobStartDate'], axis='columns', inplace=True)
# Создаю простые_заполнятели от SKlearn
cat_imputer = SimpleImputer(strategy='most_frequent') # Заполнение наиболее частыми значениями для категориальных столбцов
num_imputer = SimpleImputer(strategy='median') # Заполнение медианой для числовых столбцов
# Ищу категориальные и числовые столбцы
cat_columns, num_columns = cat_num_split(df)
# Заполняю пропуски простыми заполнятелями
df[cat_columns] = cat_imputer.fit_transform(df[cat_columns])
df[num_columns] = num_imputer.fit_transform(df[num_columns])

# Добавляю наши новые фичи, которые мы придумали
df['Carier_Level'] = CarierLevel_feature_creator_v2(df)
df['Payment_to_income'] = Payment_to_income(df)
"""

# Добавил Пластов  BEGIN
df = make_features(df)
# Добавил Пластов  END

# Сбрасываю те фичи, которые участвовали в создании новых фич
df.drop(['MonthProfit', 'MonthExpense', 'Loan_amount', 'Loan_term'], axis='columns', inplace=True)
# Смотрим на датафрейм
df.info()


Количество nan в фиче <education>: 0 штук.
Фича <education> не содержит пропусков.


Легенда по которой выполняется оцифровка уровня образования:
 {'высшее - специалист': 0.6, 'среднее профессиональное': 0.3, 'среднее': 0.2, 'неоконченное высшее': 0.4, 'бакалавр': 0.5, 'несколько высших': 0.8, 'магистр': 0.7, 'неоконченное среднее': 0.1, 'mba': 0.7, 'ученая степень': 1.0}
Количество nan в фиче <education>: 0 штук.
Фича <education> не содержит пропусков.
Исходное количество nan в фиче <employment status>: 0 штук.
Устранено: 0 штук.
Фича <employment status> не содержит пропусков.
Легенда по которой выполняется оцифровка уровня занятости:
 {'работаю по найму полный рабочий день/служу': 1.0, 'собственное дело': 1.0, 'не работаю': 0.001, 'работаю по найму неполный рабочий день': 0.5, 'студент': 0.001, 'декретный отпуск': 0.001, 'пенсионер': 0.001}
Исходное количество nan в фиче <Value>: 80 штук.
Устранено: 0 штук.
Легенда по которой выполняется оцифровка трудового стажа:
 {'10 и более лет': 1.0, '3 - 4 года': 0.35, '2 - 3 года': 0.25, '4 - 5 лет': 0.45, '5 - 6 лет': 0.55, '

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 00:00:00.0000000", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## CatBoost


In [ ]:
# Целевые переменные
banks = ['BankA_decision',
        'BankB_decision',
        'BankC_decision',
        'BankD_decision',
        'BankE_decision']

X = df.drop(banks, axis=1)
y = df[banks[0]]

features_names = list(X.columns)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)
cat_features, _ = cat_num_split(X)

# Готовим Pool для Catboost
train_data = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features,
    feature_names=features_names
)
eval_data = Pool(
    data=X_test,
    label=y_test,
    cat_features=cat_features,
    feature_names=features_names
)

model = CatBoostClassifier()


model.fit(X=train_data, eval_set=eval_data)


ftr_imp = model.feature_importances_/np.sum(model.feature_importances_)
sorted = np.argsort(ftr_imp)
ypos = np.arange(len(np.array(features_names)))
fig = plt.figure(figsize=(8, 8))
plt.barh(ypos, ftr_imp[sorted])
plt.xlim([0, 1])
plt.title('Распределение весов входных параметров')
plt.ylabel('Параметры')
plt.xlabel('Значимость')
plt.yticks(ypos, np.array(features_names)[sorted]);
plt.show()

categorical columns:	  ['BirthDate', 'education', 'employment status', 'Value', 'JobStartDate', 'Position', 'Family status', 'Goods_category', 'JobStartDate_feature_cleaner', 'CarierLevel_feature_creator_v2'] 
 len =  10
numerical columns:	  ['SkillFactory_Id', 'Gender', 'ChildCount', 'SNILS', 'Merch_code', 'Value_feature_cleaner', 'education_feature_cleaner', 'employment_status_feature_cleaner', 'CarierLevelDig_feature_creator', 'CarierVelocity_feature_creator'] 
 len =  10


TypeError: must be real number, not datetime.date

## Таблица корреляции (используются все созданные выше функции добавления новых признаков)

In [ ]:
# Загружаю датафрейм
df = pd.read_csv(
    filepath_or_buffer='SF_Mandarin_dataset_ver3_1.csv',
    sep=';',
    index_col='SkillFactory_Id',
    parse_dates=['BirthDate', 'JobStartDate']
    )
df.dropna(inplace=True) # и сказка пропадет
# Создаю признаки "Возраст" и "Стаж работы в виде числа" до заполнения пропусков
# Так как столбцы с датами нужно убивать до заполнения пропусков
df['Age'] = Age_feature_creator(df)
df['Num_value'] = Num_value_feature_creator(df)
df['Day_of_birth'] = df['BirthDate'].dt.dayofyear
df['Month_of_birth'] = df['BirthDate'].dt.month
df['Carier_Level'] = CarierLevel_feature_creator_v2(df)
df['CarierLevelDig_feature_creator'] = CarierLevelDig_feature_creator(df)
df['CarierVelocity_feature_creator'] = CarierVelocity_feature_creator(df)

# Убиваю столбцы с датами
df.drop(['BirthDate', 'JobStartDate'], axis='columns', inplace=True)
# Создаю простые_заполнятели от SKlearn
cat_imputer = SimpleImputer(strategy='most_frequent') # Заполнение наиболее частыми значениями для категориальных столбцов
num_imputer = SimpleImputer(strategy='median') # Заполнение медианой для числовых столбцов
# Перекодируем большинство категориальных признаков в числовые через cat.codes
df['BankA_decision'] = df['BankA_decision'].astype('category').cat.codes
df['BankB_decision'] = df['BankB_decision'].astype('category').cat.codes
df['BankC_decision'] = df['BankC_decision'].astype('category').cat.codes
df['BankD_decision'] = df['BankD_decision'].astype('category').cat.codes
df['BankE_decision'] = df['BankE_decision'].astype('category').cat.codes
grade = {'Неоконченное среднее': 0,
         'Среднее': 1,
         'Среднее профессиональное': 2,
         'Неоконченное высшее': 3,
         'Бакалавр': 4,
         'Высшее - специалист': 5,
         'Магистр': 6,
         'Несколько высших': 7,
         'MBA': 8,
         'Ученая степень': 9
         }
df['education'] = df['education'].astype('category').cat.rename_categories(grade)
df['Goods_category'] = df['Goods_category'].astype('category').cat.codes
df['Family status'] = df['Family status'].astype('category').cat.codes
df['employment status'] = df['employment status'].astype('category').cat.codes
#df['Value'] = df['Value'].astype('category').cat.codes

cat_columns, num_columns = cat_num_split(df)
# Заполняю пропуски простыми заполнятелями
df[cat_columns] = cat_imputer.fit_transform(df[cat_columns])
df[num_columns] = num_imputer.fit_transform(df[num_columns])
#
df['Payment_to_income'] = Payment_to_income(df)


corr = df[num_columns].corr()
corr.style.background_gradient(cmap='coolwarm')